In [39]:
import lmdb
from pathlib import Path
import pickle
from fairchem.core.common.utils import pyg2_data_transform

In [38]:
db_path = "/Users/curtischong/Documents/dev/joule/src/fairchem/data/s2ef_test_lmdbs/test_data/s2ef/all/test_id/data.0000.lmdb"

def connect_db(lmdb_path: Path | None = None) -> lmdb.Environment:
    return lmdb.open(
        str(lmdb_path),
        subdir=False,
        readonly=True,
        lock=False,
        readahead=True,
        meminit=False,
        max_readers=1,
    )

cur_env = connect_db(db_path)

In [34]:
# If "length" encoded as ascii is present, use that
length_entry = cur_env.begin().get("length".encode("ascii"))
if length_entry is not None:
    num_entries = pickle.loads(length_entry)
else:
    # Get the number of stores data from the number of entries
    # in the LMDB
    num_entries = cur_env.stat()["entries"]


In [35]:
num_entries

625

In [40]:
el_idx = 1
datapoint_pickled = (
    cur_env
    .begin()
    .get(f"{el_idx}".encode("ascii"))
)
data_object = pyg2_data_transform(pickle.loads(datapoint_pickled))

In [44]:
data_object.tags

tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 2])